1. Dataset has been downloaded and saved

2. Import required Libraries

In [7]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

3. Read dataset and do preprocessing

In [8]:
df = pd.read_csv(
    './dataset/spam.csv',
    delimiter = ',',
    encoding = 'latin-1'
)

In [9]:
df.drop(
    ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
    axis = 1,
    inplace = True
)

In [10]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size = 0.15
)

In [12]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen = max_len)

5. Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [13]:
def RNN():
    inputs = Input(name = 'inputs', shape = [max_len])
    layer = Embedding(max_words, 50, input_length = max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs = inputs, outputs = layer)
    return model

4. Create Model

In [14]:
model = RNN()

6. Compile the model

In [15]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

7. Fit the model

In [16]:
model.fit(
    sequences_matrix,
    Y_train,
    batch_size = 128,
    epochs=10,
    validation_split = 0.2,
    callbacks=[EarlyStopping(monitor = 'val_loss', min_delta = 0.0001)])

Epoch 1/10
30/30 [==============================] - 8s 149ms/step - loss: 0.3363 - accuracy: 0.8683 - val_loss: 0.1425 - val_accuracy: 0.9536
Epoch 2/10
30/30 [==============================] - 4s 142ms/step - loss: 0.0888 - accuracy: 0.9786 - val_loss: 0.0562 - val_accuracy: 0.9895


8. Save the model

In [17]:
model.save('./spam.h5')

9. Test the model

In [18]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen = max_len)

In [19]:
accr = model.evaluate(test_sequences_matrix, Y_test)

27/27 [==============================] - 1s 24ms/step - loss: 0.0704 - accuracy: 0.9761


In [20]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.070
  Accuracy: 0.976
